In [1]:
import os
import pandas as pd
from io import StringIO


SCORE_DIR = "data/scores"

/Users/Anaqi_Amir/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Getting all the games
box_scores = os.listdir(SCORE_DIR)
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [3]:
from bs4 import BeautifulSoup

def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [4]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [5]:
def read_line_score(soup):
    line_score = pd.read_html(StringIO(str(soup)), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols

    line_score = line_score[["team", "total"]]

    return line_score

In [6]:
def read_stats(soup, team, stat):
    df = pd.read_html(StringIO(str(soup)), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [7]:
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)

    line_score = read_line_score(soup) # line score
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")
        del advanced[advanced.columns[0]] # deleting mp in advanced

        summary = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        summary.index = summary.index.str.lower()

        # Initializing headers (run once only)
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols] # Turning them into headers

        summaries.append(summary)

    # Append game summary to list of summaries of all games
    summary = pd.concat(summaries, axis=1).T

    summary = summary.reset_index(drop=True)
    line_score = line_score.reset_index(drop=True)

    game = pd.concat([summary, line_score], axis=1)

    # Separating stats of away teams
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns = "away_" + game_opp.columns
    game.columns = "home_" + game.columns

    # Season and dates
    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    # Target column (wrt to home team)
    full_game["home_won"] = full_game["home_total"] > full_game["away_total"]
    games.append(full_game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 1319
200 / 1319
300 / 1319
400 / 1319
500 / 1319
600 / 1319
700 / 1319
800 / 1319
900 / 1319
1000 / 1319
1100 / 1319
1200 / 1319
1300 / 1319


In [8]:
# Putting it into a dataframe
games_df = pd.concat(games, ignore_index=True)

# Dropping unncessary columns
games_df.drop(['away_index','away_mp'],axis=1,inplace=True)

# Rename mp
games_df.rename(columns={'home_mp':'mp'},inplace=True)

# Rearranging columns for readability
column_to_move = games_df.pop("season")
games_df.insert(0, "season", column_to_move)

column_to_move = games_df.pop("date")
games_df.insert(1, "date", column_to_move)

column_to_move = games_df.pop("home_team")
games_df.insert(2, "home_team", column_to_move)

column_to_move = games_df.pop("home_total")
games_df.insert(3, "home_total", column_to_move)

column_to_move = games_df.pop("away_team")
games_df.insert(4, "away_team", column_to_move)

column_to_move = games_df.pop("away_total")
games_df.insert(5, "away_total", column_to_move)

games_df.head()

,season,date,home_team,home_total,away_team,away_total,mp,home_fg,home_fga,home_fg%,...,away_drb%,away_trb%,away_ast%,away_stl%,away_blk%,away_tov%,away_usg%,away_ortg,away_drtg,home_won
0,2024,2024-02-13,BOS,118,BRK,110,240.0,43.0,83.0,0.518,...,81.1,48.7,71.8,6.3,6.5,11.7,100.0,115.3,123.7,True
1,2024,2024-02-13,BRK,110,BOS,118,240.0,39.0,79.0,0.494,...,82.1,51.3,72.1,6.3,2.2,12.3,100.0,123.7,115.3,False
2,2024,2023-11-11,MIA,117,ATL,109,240.0,41.0,84.0,0.488,...,82.5,55.2,60.5,6.0,13.0,18.1,100.0,109.3,117.3,True
3,2024,2023-11-11,ATL,109,MIA,117,240.0,38.0,87.0,0.437,...,68.1,44.8,56.1,10.0,9.8,13.6,100.0,117.3,109.3,False
4,2024,2024-04-11,GSW,100,POR,92,240.0,36.0,77.0,0.468,...,74.4,51.6,62.9,9.9,7.1,10.3,100.0,101.2,110.0,True


In [9]:
# Sort according to dates
games_df.sort_values("date",inplace=True,ascending=True)
games_df.reset_index(inplace=True,drop=True)

In [10]:
# Save as csv file
games_df.to_csv("nba_games_2024.csv",index=False)